In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import torch

# Загрузка предобученной модели и токенизатора BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Установка режима оценки модели
model.eval()

C:\Users\zhest\PycharmProjects\LSINewsGroup\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [3]:
categories = ['rec.autos', 'comp.graphics', 'sci.space']
newsgroup = fetch_20newsgroups(categories=categories, remove=('headers', 'footers', 'quotes'))
X_train, X_test, y_train, y_test = train_test_split(newsgroup.data, newsgroup.target, test_size=0.33)

In [4]:
def get_bert_embeddings(data):
    embeddings = []
    for text in data:
        # Токенизация текста
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        # Получение выходных данных модели
        with torch.no_grad():
            outputs = model(**inputs)
        # Получение векторного представления первого токена ([CLS])
        embeddings.append(outputs.last_hidden_state[:, 0, :].squeeze().numpy())
    return embeddings

# Получение векторных представлений
train_embeddings = get_bert_embeddings(X_train)
test_embeddings = get_bert_embeddings(X_test)

# Вывод размерности первого эмбеддинга
print("Размерность эмбеддинга:", test_embeddings[0].shape)

# Вывод части эмбеддинга (например, первые 50 элементов)
print("Часть эмбеддинга (первые 10 элементов):", test_embeddings[0][:50])

Размерность эмбеддинга: (768,)
Часть эмбеддинга (первые 10 элементов): [-1.9891600e-01  1.5745599e-02  5.9442866e-01 -8.0169231e-02
  5.8570422e-02 -3.9691585e-01  5.3460062e-01  5.4816628e-01
  5.1852625e-02 -7.0319724e-01 -3.1497896e-01  1.3438076e-01
 -1.6629755e-01 -4.2011961e-06  4.2172956e-01  1.9615266e-01
 -9.9871039e-02  5.7500803e-01  9.5272407e-02  3.2930346e-03
 -7.1591325e-02 -3.0659835e-03  7.0639871e-02 -2.1891215e-01
  7.0447825e-02 -2.1799985e-01 -3.2501486e-01  6.4198747e-02
 -3.6999023e-01 -8.5328609e-02 -2.3995636e-01  3.5967371e-01
 -3.1141404e-02  1.9907460e-01  7.2272575e-01 -2.4487387e-02
 -1.5311117e-01 -9.8912686e-02  2.1869126e-01 -2.2832084e-01
 -4.3043691e-01  3.3968508e-02  1.9507127e-01  1.7244959e-01
 -8.9092784e-02 -4.8968382e-04 -4.2578268e+00 -2.2965886e-01
 -2.8059554e-01 -1.8890041e-01]


In [5]:
# Создание и обучение классификатора Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(train_embeddings, y_train)

RandomForestClassifier(random_state=42)

In [6]:
# Предсказание на тестовом наборе
test_preds = clf.predict(test_embeddings)

# Вычисление F1-score
f1 = f1_score(y_test, test_preds, average='micro')
print(f"F1-micro score: {f1}")

F1-micro score: 0.8495726495726496
